# Creating a Custom LUME-model for probabilistic models


In [1]:
from torch.distributions.normal import Normal
import torch
from lume_model.models.prob_model_base import ProbModelBaseModel
from lume_model.variables import ScalarVariable, DistributionVariable

## Create a model that returns a list of predictions

In [2]:
class ExampleModel(ProbModelBaseModel):
    """Model returns a list of predictions for each output"""

    def _get_predictions(self, input_dict):
        """
        This method implements the required abstract method for this class.
        It takes the input_dict and returns a dict of output names to distributions.
        """
        # Just generate random output here for this example
        # but typically this is where you would adjust the input if needed and
        # call your model on the input
        output_dict = {
            "output1": torch.rand(5),
            "output2": torch.rand(10),
        }
        return self._create_output_dict(output_dict)

    def _create_output_dict(self, output):
        """This method is not required by the abstract class but typically
        needed to create a distribution type output for each output
        name from the list of predictions that the model returns.
        """
        output_dict = {}
        for k, v in output.items():
            output_dict[k] = Normal(v.mean(axis=0), torch.sqrt(v.var(axis=0)))
        return output_dict

### Model Instantiation and Execution
Instantiation requires specification of the input and output variables of the model.

In [3]:
input_variables = [
    ScalarVariable(name="input1", default_value=0.1),
    ScalarVariable(name="input2", default_value=0.2, value_range=[0.0, 1.0]),
]
output_variables = [
    DistributionVariable(name="output1"),
    DistributionVariable(name="output2"),
]

m = ExampleModel(input_variables=input_variables, output_variables=output_variables)

In [4]:
input_dict = {
    "input1": 0.3,
    "input2": 0.6,
}
out = m.evaluate(input_dict)

In [5]:
out

{'output1': Normal(loc: 0.4858802855014801, scale: 0.3480694591999054),
 'output2': Normal(loc: 0.5287243127822876, scale: 0.28792139887809753)}

In [6]:
out["output1"].mean, out["output1"].variance

(tensor(0.4859), tensor(0.1212))